# SBV2学習後に実行する後処理用のGoogle Colabスクリプト
公式の下記の実装を使わせていただいております。

https://github.com/litagin02/Style-Bert-VITS2/blob/master/colab.ipynb

https://github.com/litagin02/Style-Bert-VITS2/blob/master/library.ipynb


「ランタイム」→「ランタイムのタイプを変更」

から、「T4 GPU」に変更してから、「Shift+Enter」で上からセルを実行してください。

# Google Driveのマウント
認証が入るので、最初のセルに置くのをオススメします。

In [ ]:
#Google Driveのフォルダをマウント（認証入る）
from google.colab import drive
drive.mount('/content/drive')

# 必要パッケージのインストール

In [ ]:
# このセルを実行して環境構築してください。
# エラーダイアログ「WARNING: The following packages were previously imported in this runtime: [pydevd_plugins]」が出るが「キャンセル」を選択して続行してください。

import os

os.environ["PATH"] += ":/root/.cargo/bin"

!curl -LsSf https://astral.sh/uv/install.sh | sh
!git clone https://github.com/litagin02/Style-Bert-VITS2.git
%cd Style-Bert-VITS2/
!uv pip install --system -r requirements.txt
!uv pip install --system style-bert-vits2
!python initialize.py --skip_default_models

# 各種パスやモデル名の指定

In [ ]:
model_name = "amitaro"

# Google Driveでのパスを特定する
import glob
import os
drive_path = os.path.dirname(glob.glob('/content/drive/MyDrive/colab_SBV2train_sample/SBV2-evaluation.ipynb', recursive=True)[0])
print(drive_path)

# 学習に必要なファイルや途中経過が保存されるディレクトリ
dataset_root = f"{drive_path}/Data"

# 学習結果（音声合成に必要なファイルたち）が保存されるディレクトリ
assets_root = f"{drive_path}/model_assets"

import yaml
with open("configs/paths.yml", "w", encoding="utf-8") as f:
    yaml.dump({"dataset_root": dataset_root, "assets_root": assets_root}, f)

# モデルの評価

学習後、複数個保存された途中経過の重みが存在するため、そのタイミングの重みを利用するのが良さそうか判定するスクリプトを実行する

評価が高くて、学習の後半の重みファイルを利用することをオススメするが、あくまでもベンチマーク的な結果であるため、最終的には各々が聴き比べをして、良いモデルを選定することをオススメする。

In [ ]:
!python speech_mos.py -m {model_name}

# 実際に合成音声を試す

In [ ]:
# 評価した結果、良さそうなモデルのファイル名を指定する
safetensor = "amitaro_e18_s6000.safetensors"

In [ ]:
# BERTモデルをロード（ローカルに手動でダウンロードする必要はありません）

from style_bert_vits2.nlp import bert_models
from style_bert_vits2.constants import Languages


bert_models.load_model(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")
bert_models.load_tokenizer(Languages.JP, "ku-nlp/deberta-v2-large-japanese-char-wwm")


from pathlib import Path

# 学習したモデルのパスを指定する
model_file = f"{model_name}/{safetensor}"
config_file = f"{model_name}/config.json"
style_file = f"{model_name}/style_vectors.npy"


from style_bert_vits2.tts_model import TTSModel

assets_root_path = Path(assets_root)

# SBV2のモデル自体をロードする
model = TTSModel(
    model_path=assets_root_path / model_file,
    config_path=assets_root_path / config_file,
    style_vec_path=assets_root_path / style_file,
    device="cpu",
)


In [ ]:
from IPython.display import Audio, display

# 音声合成を実施する
sr, audio = model.infer(text="そうだね！楽しみだね！", style = "Neutral")
display(Audio(audio, rate=sr))

In [ ]:
#作成したスタイル一つ目　conversation（普通の会話音声）
sr, audio = model.infer(text="えー！ほんとかなあ", style = "conversation")
display(Audio(audio, rate=sr))

In [ ]:
# 作成したスタイル二つ目　whisper（囁き声）
sr, audio = model.infer(text="こんにちは、今日もお疲れ様。", style = "whisper")
display(Audio(audio, rate=sr))

# 実行後Colabノードブックのランタイムを削除する

特に、有料版のColabでは、接続時間に応じてコンピューティングユニットが消費されてしまうので、節約のため、学習完了したら即座にColabのランタイムを削除する必要がある。

下記セルにおいて、`FLAG = True`として、実行すると、ランタイムが接続解除される

In [ ]:
# ノートブックの解放
FLAG = False

from google.colab import runtime

if FLAG:
    runtime.unassign()